<a href="https://www.kaggle.com/code/akashrayhan/yolo-v7-pose-keypoints?scriptVersionId=112482642" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import YOLO v7 code and pre-trained weights for pose estimation

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
sys.path.append("/kaggle/input/yolov7-lib/yolov7-main")
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import transforms
import numpy as np
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/yolov7-weights'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yolov7-weights/yolov7-e6.pt
/kaggle/input/yolov7-weights/yolov7-d6.pt
/kaggle/input/yolov7-weights/yolov7x.pt
/kaggle/input/yolov7-weights/yolov7-e6e.pt
/kaggle/input/yolov7-weights/yolov7-tiny.pt
/kaggle/input/yolov7-weights/zidane.jpeg
/kaggle/input/yolov7-weights/yolov7-w6-pose.pt
/kaggle/input/yolov7-weights/yolov7.pt
/kaggle/input/yolov7-weights/yolov7-w6.pt
/kaggle/input/yolov7-weights/yolov7-mask.pt


#### Initialize the pre-trained weights and model

In [2]:
# load weights 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# load pre-trained weights
weigths = torch.load('/kaggle/input/yolov7-weights/yolov7-w6-pose.pt', map_location=device)
model = weigths['model']
_ = model.float().eval()
if torch.cuda.is_available():
    model.half().to(device)

#### Getting Keypoints for each images

In [3]:
import gc
key_points_dict = {}
images_path = '/kaggle/input/dataset/images/'
images = os.listdir(images_path)
#iterate over the images
for img in images:
    #no_grad() to my model tells PyTorch that I don't want to store any previous computations, thus freeing my GPU space.
    with torch.no_grad():
        
        ## Transform image from numpy to torch format    
        image = cv2.imread(images_path+img)
        image = letterbox(image, 960, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        
        if torch.cuda.is_available():
            image = image.half().to(device)   
        output, _ = model(image)
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        
    keypoints = output_to_keypoint(output)
    # iterate for multiple figures in an image
    for idx in range(keypoints.shape[0]):
        keypoints_list = keypoints[idx,7:].tolist()
    key_points_dict[img] = keypoints_list
        
    del image, output
    torch.cuda.empty_cache()
    gc.collect()
        


/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
key_points_df = pd.DataFrame(key_points_dict.items(),columns=["image_id","key_points"])

In [5]:
key_points_df.head(5)

,image_id,key_points
0,Image_5576.jpg,"[439.25, 292.0, 0.9951171875, 482.75, 298.75, ..."
1,Image_8010.jpg,"[576.0, 242.75, 0.8505859375, 582.5, 236.875, ..."
2,Image_5775.jpg,"[435.5, 259.0, 0.986328125, 441.25, 251.0, 0.8..."
3,Image_7411.jpg,"[434.75, 364.0, 0.994140625, 431.75, 333.0, 0...."
4,Image_4172.jpg,"[187.375, 343.25, 0.99658203125, 179.0, 320.5,..."


In [6]:
header_list = ["image_id", "labels"]
labels_df = pd.read_csv('/kaggle/input/labels/labels.csv',names=header_list)

In [7]:
labels_df.head(5)

,image_id,labels
0,Image_1.jpg,sitting
1,Image_4.jpg,sleeping
2,Image_6.jpg,sleeping
3,Image_15.jpg,sleeping
4,Image_33.jpg,sitting


In [8]:
# merging the keypoints_df and labels_df on the common column'image_id'
unbalanced_keypoints_df = pd.merge(left=key_points_df,right=labels_df,how='inner',on='image_id')

In [9]:
unbalanced_keypoints_df.head(5)

,image_id,key_points,labels
0,Image_5576.jpg,"[439.25, 292.0, 0.9951171875, 482.75, 298.75, ...",sleeping
1,Image_8010.jpg,"[576.0, 242.75, 0.8505859375, 582.5, 236.875, ...",running
2,Image_5775.jpg,"[435.5, 259.0, 0.986328125, 441.25, 251.0, 0.8...",sitting
3,Image_7411.jpg,"[434.75, 364.0, 0.994140625, 431.75, 333.0, 0....",sleeping
4,Image_4172.jpg,"[187.375, 343.25, 0.99658203125, 179.0, 320.5,...",sleeping


In [10]:
#export unbalanced_keypoints_df to a csv file 
unbalanced_keypoints_df.to_csv('unbalanced_keypoints.csv',index=False)

In [11]:
show_imbalanced_data = unbalanced_keypoints_df.groupby(['labels'])['labels'].count()
print(show_imbalanced_data)

labels
running     190
sitting     379
sleeping    700
Name: labels, dtype: int64


Sleeping label has 700 examples, sitting has 379, running has 190. As we can see running examples are comparatively low than other examples. So the dataset is imbalanced, it might cause low generalization in training

In [12]:
# drop the image_id column
unbalanced_keypoints_df.drop('image_id', axis=1, inplace=True)
#separate individual classes data
running_data = unbalanced_keypoints_df[unbalanced_keypoints_df.labels == 'running']
sitting_data = unbalanced_keypoints_df[unbalanced_keypoints_df.labels == 'sitting']
sleeping_data = unbalanced_keypoints_df[unbalanced_keypoints_df.labels == 'sleeping']


# Oversampling Minority Class
Oversampling can be defined as adding more copies of the minority class. 
We will use the resampling module from Scikit-Learn to randomly replicate samples from the minority class.

In [13]:
from sklearn.utils import resample
running_upsample = resample(running_data,
                            replace=True,
                            n_samples=len(sleeping_data),#number of examples in majority class
                            random_state=42)

sitting_upsample = resample(sitting_data,
                            replace=True,
                            n_samples=len(sleeping_data),
                            random_state=42)

In [14]:
unbalanced_df_upsample = pd.concat([running_upsample,sitting_upsample,sleeping_data])

In [15]:
unbalanced_df_upsample.groupby(['labels'])['labels'].count()

labels
running     700
sitting     700
sleeping    700
Name: labels, dtype: int64

In [16]:
unbalanced_df_upsample.to_csv('balanced_keypoints_upsample.csv',index=False)

## SMOTE
#### SMOTE or Synthetic Minority Oversampling Technique is a popular algorithm to creates synthetic observations of the minority class.

In [17]:
# unpacking the keypoints value in list
unbalanced_keypoints_df['key_points'][:2].apply(pd.Series)

,0,1,2,3,4,...,46,47,48,49,50
0,439.25,292.00,0.995117,482.75,298.750,...,377.0,0.034363,144.875,375.0,0.031494
1,576.00,242.75,0.850586,582.50,236.875,...,534.0,0.889648,713.000,535.5,0.886719


In [18]:
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

features = unbalanced_keypoints_df['key_points'].apply(pd.Series)
targets = lb.fit_transform(unbalanced_keypoints_df['labels'])

sm = SMOTE(random_state=42)
key_points_resample, labels_resample = sm.fit_resample(features, targets)

In [19]:
key_points_resample_df = pd.DataFrame(key_points_resample)
labels_resample_df = pd.DataFrame(labels_resample)

In [20]:
balanced_keypoints_smote = pd.concat([key_points_resample_df, labels_resample_df], axis=1)

In [21]:
balanced_keypoints_smote .to_csv('balanced_keypoints_smote .csv',index=False)